**Run 25 July - RAG. Phase Z

RAG for geetha vahini.pdf




# Part 1: (Aggressive) Installation.

IMPORTANT: follow these steps precisely:
1.	Run this code block.
2.	After it completes, go to Runtime -> Disconnect and delete runtime.
3.	Once the runtime restarts, RUN THIS CODE BLOCK AGAIN.
4.	After this block finishes its second run, you can safely proceed to the next sections.

In [1]:
# --- Installation Block ---
print("Starting library installations and upgrades for RAG...")

# Aggressively uninstall to ensure a clean slate for core libraries
# This helps prevent conflicts with pre-installed Colab packages or previous runs.
!pip uninstall -y torch torchvision torchaudio transformers accelerate bitsandbytes trl peft datasets xformers langchain langchain-community pypdf chromadb sentence-transformers

# Clear relevant caches
# Clearing bitsandbytes cache can help resolve issues with 4-bit quantization.
print("Clearing bitsandbytes cache...")
!rm -rf ~/.cache/bitsandbytes
# Clearing Hugging Face cache can help if model downloads were corrupted.
print("Clearing Hugging Face cache...")
!rm -rf ~/.cache/huggingface/hub/*

# Install PyTorch and Torchvision specifically for CUDA 12.1 (common in Colab)
# This is crucial for GPU acceleration and compatibility.
print("Installing PyTorch and Torchvision for CUDA 12.1...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install core Hugging Face libraries
# `transformers`: For loading LLMs and tokenizers.
# `accelerate`: For efficient model loading and GPU usage.
# `bitsandbytes`: For 4-bit quantization (memory efficiency).
# `trl`: For Transformer Reinforcement Learning (though not directly used in Phase 1, it's common for fine-tuning).
# `peft`: Parameter-Efficient Fine-Tuning (for LoRA adapters, used later).
# `datasets`: For handling datasets efficiently.
print("Installing transformers, accelerate, bitsandbytes, trl, peft, datasets...")
!pip install transformers accelerate bitsandbytes "trl==0.8.6" peft datasets

# Install RAG-specific libraries
# `langchain`: Orchestration framework for building LLM applications.
# `langchain-community`: Contains various integrations, including PDF loaders and vector stores.
# `pypdf`: Library for reading PDF files.
# `chromadb`: The specific vector database we'll use.
# `sentence-transformers`: For the embedding model.
print("Installing langchain, langchain-community, pypdf, chromadb, sentence-transformers...")
!pip install langchain langchain-community pypdf chromadb sentence-transformers

# xformers is optional, uncomment if you want to try it. It can optimize attention mechanisms
# but sometimes causes installation/compatibility issues. Not strictly necessary for basic RAG.
# !pip install xformers

print("\nLibrary installation complete.")
print("IMPORTANT: Please follow the instructions above about restarting the runtime.")


Starting library installations and upgrades for RAG...
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Found existing installation: transformers 4.54.0
Uninstalling transformers-4.54.0:
  Successfully uninstalled transformers-4.54.0
Found existing installation: accelerate 1.9.0
Uninstalling accelerate-1.9.0:
  Successfully uninstalled accelerate-1.9.0
Found existing installation: bitsandbytes 0.46.1
Uninstalling bitsandbytes-0.46.1:
  Successfully uninstalled bitsandbytes-0.46.1
Found existing installation: trl 0.8.6
Uninstalling trl-0.8.6:
  Successfully uninstalled trl-0.8.6
Found existing installation: peft 0.16.0
Uninstalling pe

# Part 2: Phase 1: load pdf, chunk it, vectorize chunks, store chunk embeddings in chroma db. Embedding model: BAAI/bge-small-en-v1.5 (a Sentence-BERT model)

Step Description	Component/Model Used	Vendor/Library/Database Used
Phase 1: Knowledge Ingestion (Building Your Searchable Library)
1. Document Loading & Text Extraction	N/A (Text extraction logic)	PyPDFLoader (from langchain-community)
2. Text Chunking	N/A (Text splitting algorithm)	RecursiveCharacterTextSplitter (from langchain.text_splitter)
3. Embedding (Vectorization)	Embedding Model: BAAI/bge-small-en-v1.5 (a Sentence-BERT model)	Hugging Face (model hub), sentence-transformers library
4. Vector Store (Storage & Indexing)	N/A (Database technology)	ChromaDB (from langchain-community.vectorstores)


In [2]:
import os
from google.colab import drive # For Google Drive mounting
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# --- Configuration ---
# Ensure this DATA_PATH matches the folder where your PDF is located in Google Drive
DATA_PATH = "/content/drive/MyDrive/266_fp_ph_z"
PDF_FILE_NAME = "geetha_vahini.pdf"
PDF_FILE_PATH = os.path.join(DATA_PATH, PDF_FILE_NAME) # CORRECT. gv.pdf is in the original fp phz folder

CHROMA_DB_DIR = "/content/drive/MyDrive/266_fp_ph_z/ph_z_rag" # NEW RAG DIR FOR RAG SPECIFIC STORES AND OUTPUT
CHROMA_DB_PATH = os.path.join(CHROMA_DB_DIR, "chroma_db") # Path to store Chroma DB persistently

# --- Mount Google Drive (if not already mounted in this session) ---
print("Mounting Google Drive...")
if not os.path.exists('/content/drive/MyDrive'):
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully!")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        print("Please ensure you are running this in a Google Colab environment and authorize Drive access.")
        exit()
else:
    print("Google Drive already mounted.")

# Ensure the RAG data directory exists
os.makedirs(DATA_PATH, exist_ok=True)
print(f"Ensured RAG data directory exists: {DATA_PATH}")

# --- Check for PDF existence ---
if not os.path.exists(PDF_FILE_PATH):
    print(f"Error: PDF file '{PDF_FILE_NAME}' not found at {PDF_FILE_PATH}.")
    print("Please ensure your 'Geetha Vahini.pdf' is uploaded to this path.")
    exit()
else:
    print(f"PDF file '{PDF_FILE_NAME}' found. Proceeding.")

# --- Step 1: Load PDF Document ---
print(f"\nStep 1: Loading PDF document from: {PDF_FILE_PATH}")
try:
    loader = PyPDFLoader(PDF_FILE_PATH)
    documents = loader.load()
    print(f"Loaded {len(documents)} pages from the PDF.")
except Exception as e:
    print(f"Error loading PDF: {e}")
    print("Please ensure the PDF file is valid and accessible.")
    exit()

# --- Step 2: Split Text into Chunks ---
print("\nStep 2: Splitting documents into chunks...")
# Recommended chunk_size and chunk_overlap for general QA
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Max characters per chunk
    chunk_overlap=200,    # Overlap between chunks to maintain context
    length_function=len,
    add_start_index=True, # Adds metadata about where the chunk came from
)
chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} text chunks.")
if chunks:
    print(f"Example chunk (first 200 chars): {chunks[0].page_content[:200]}...")
    print(f"Source page for example chunk: {chunks[0].metadata.get('page', 'N/A')}")
else:
    print("No chunks were created. Check PDF content or chunking parameters.")

# --- Step 3 & 4: Create Embeddings and Build Vector Store (ChromaDB) ---
print("\nStep 3 & 4: Creating embeddings and building/loading vector store (this may take some time)...")
# Choose an embedding model. 'BAAI/bge-small-en-v1.5' is a good balance of quality and speed.
embedding_model_name = "BAAI/bge-small-en-v1.5"
embeddings = SentenceTransformerEmbeddings(model_name=embedding_model_name)

try:
    # Check if a Chroma DB already exists at the path to avoid re-embedding
    if os.path.exists(CHROMA_DB_PATH) and os.listdir(CHROMA_DB_PATH):
        print(f"Loading existing Chroma DB from {CHROMA_DB_PATH}...")
        vectorstore = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embeddings)
        print("Existing Chroma DB loaded.")
    else:
        print(f"Creating new Chroma DB and persisting to {CHROMA_DB_PATH}...")
        if not chunks:
            print("Cannot create vector store: No chunks available. Please check previous steps.")
            exit()
        vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=CHROMA_DB_PATH)
        vectorstore.persist() # Explicitly persist the database to disk
        print("New Chroma DB created and persisted.")
except Exception as e:
    print(f"Error creating/loading ChromaDB: {e}")
    print("Please check your disk space, permissions, or ensure the embedding model loaded correctly.")
    exit()

print("\nPhase 1: Knowledge Ingestion complete. Vector store is ready for retrieval!")


Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully!
Ensured RAG data directory exists: /content/drive/MyDrive/266_fp_ph_z
PDF file 'geetha_vahini.pdf' found. Proceeding.

Step 1: Loading PDF document from: /content/drive/MyDrive/266_fp_ph_z/geetha_vahini.pdf
Loaded 146 pages from the PDF.

Step 2: Splitting documents into chunks...
Created 508 text chunks.
Example chunk (first 200 chars): GEETHA VAHINI
(The Divine Gospel)
by
Bhagawan Sri Sathya Sai Baba
SRI SATHYA SAI BOOKS & PUBLICATIONS TRUST
Prasanthi Nilayam - 515 134
Anantapur District, Andhra Pradesh, India.
Grams: BOOK TRUST    ...
Source page for example chunk: 0

Step 3 & 4: Creating embeddings and building/loading vector store (this may take some time)...


/tmp/ipython-input-2-2612464073.py:74: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading existing Chroma DB from /content/drive/MyDrive/266_fp_ph_z/ph_z_rag/chroma_db...


/tmp/ipython-input-2-2612464073.py:80: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embeddings)


Existing Chroma DB loaded.

Phase 1: Knowledge Ingestion complete. Vector store is ready for retrieval!


# Part 3: ACCESSIBILITY CHECKS (unit test file, model adapters, hugging face login, base model Llama-3-8-B-Instruct can be loaded), chroma db vector store can be accessed.


 Access to unit test file: unit_test_passage_questions_clean.jsonl. It is in jsonl format
 Successful login to Hugging Face
 Access to fine tuned adapters
 Base MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct" can be loaded
 Chroma DB vector store can be loaded

In [3]:
#######
## ACCESSIBILITY CHECKS
# Access to unit test file: unit_test_passage_questions_clean.jsonl. It is in jsonl format
# Successful login to Hugging Face
# Access to fine tuned adapters
# Base MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct" can be loaded
# Chroma DB vector store can be loaded
########################

import os
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from huggingface_hub import login
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from google.colab import drive

# --- Configuration ---
ROOT_DIR_ADAPTERS = "/content/drive/MyDrive/266_fp_ph_z"
FINE_TUNED_ADAPTERS_FOLDER_NAME = "llama3_8b_qa_finetuned_adapters_standard_hf"
FINE_TUNED_ADAPTERS_PATH = os.path.join(ROOT_DIR_ADAPTERS, FINE_TUNED_ADAPTERS_FOLDER_NAME) #CORRECT. ADAPTERS ARE IN THE ORIGINAL ROOT DIR FOR PHASE Z
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"


CHROMA_DB_DIR = "/content/drive/MyDrive/266_fp_ph_z/ph_z_rag" # NEW RAG DIR FOR RAG SPECIFIC STORES AND OUTPUT
CHROMA_DB_PATH = os.path.join(CHROMA_DB_DIR, "chroma_db") # Path to store Chroma DB persistently


UNIT_TEST_FILE_NAME = "unit_test_passage_questions_clean_ph_z.jsonl"
UNIT_TEST_FILE_PATH = os.path.join(CHROMA_DB_DIR, UNIT_TEST_FILE_NAME)

# --- Mount Google Drive ---
print("Mounting Google Drive...")
if not os.path.exists('/content/drive/MyDrive'):
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully!")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        print("Please ensure you are running this in a Google Colab environment and authorize Drive access.")
        exit()
else:
    print("Google Drive already mounted.")

# --- Check for Unit Test File Existence ---
print(f"\nChecking for existence of Unit Test File: {UNIT_TEST_FILE_PATH}")
if os.path.exists(UNIT_TEST_FILE_PATH):
    print(f"SUCCESS: Unit test file '{UNIT_TEST_FILE_NAME}' found at {UNIT_TEST_FILE_PATH}.")
    try:
        with open(UNIT_TEST_FILE_PATH, 'r', encoding='utf-8') as f:
            first_line = f.readline()
            json.loads(first_line) # Try to parse the first line to check if it's valid JSON
        print("SUCCESS: Unit test file appears to be valid JSONL format.")
    except Exception as e:
        print(f"WARNING: Unit test file found, but could not parse as JSONL. Error: {e}")
else:
    print(f"ERROR: Unit test file '{UNIT_TEST_FILE_NAME}' NOT found at {UNIT_TEST_FILE_PATH}.")
    print("Please ensure it is uploaded to your RAG data directory.")


# --- Hugging Face Login ---
print("\nAttempting Hugging Face Hub login...")
try:
    login()
    print("SUCCESS: Hugging Face login successful!")
except Exception as e:
    print(f"ERROR: Hugging Face login failed: {e}")
    print("Please ensure you have accepted the Llama 3 license and pasted a valid token.")


# --- Check for Fine-tuned Adapters Existence ---
print(f"\nChecking for existence of Fine-tuned Adapters: {FINE_TUNED_ADAPTERS_PATH}")
if os.path.exists(os.path.join(FINE_TUNED_ADAPTERS_PATH, "adapter_config.json")) and \
   os.path.exists(os.path.join(FINE_TUNED_ADAPTERS_PATH, "adapter_model.safetensors")):
    print("SUCCESS: Fine-tuned adapter files found.")
else:
    print(f"ERROR: Fine-tuned adapter files NOT found in {FINE_TUNED_ADAPTERS_PATH}.")
    print("Please ensure they were copied/extracted correctly.")


# --- Attempt to Load Base Model ---
print(f"\nAttempting to load Base Model: {MODEL_NAME}...")
try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
    base_model_loaded = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
    )
    tokenizer_loaded = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    print("SUCCESS: Base model and tokenizer loaded.")
    # Clean up to free memory if this is just a check
    del base_model_loaded
    del tokenizer_loaded
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
except Exception as e:
    print(f"ERROR: Could not load Base Model. This might be due to HF login, network issues, or model availability. Error: {e}")


# --- Attempt to Load ChromaDB Vector Store ---
print(f"\nChecking for existence and attempting to load ChromaDB from: {CHROMA_DB_PATH}")
try:
    if os.path.exists(CHROMA_DB_PATH) and os.listdir(CHROMA_DB_PATH):
        # A lightweight embedding function is sufficient for just loading the DB structure
        temp_embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")
        vectorstore_loaded = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=temp_embeddings)
        # Try a dummy query to ensure it's functional
        count = vectorstore_loaded._collection.count()
        print(f"SUCCESS: ChromaDB found and loaded. Contains {count} documents.")
        # Clean up
        del vectorstore_loaded
    else:
        print(f"ERROR: ChromaDB directory not found or is empty at {CHROMA_DB_PATH}.")
        print("Please ensure Phase 1 (Knowledge Ingestion) was completed successfully.")
except Exception as e:
    print(f"ERROR: Could not load ChromaDB. Error: {e}")
    print("This might indicate corruption or an issue during its creation in Phase 1.")

print("\n--- All accessibility checks complete. ---")


Mounting Google Drive...
Google Drive already mounted.

Checking for existence of Unit Test File: /content/drive/MyDrive/266_fp_ph_z/ph_z_rag/unit_test_passage_questions_clean_ph_z.jsonl
SUCCESS: Unit test file 'unit_test_passage_questions_clean_ph_z.jsonl' found at /content/drive/MyDrive/266_fp_ph_z/ph_z_rag/unit_test_passage_questions_clean_ph_z.jsonl.
SUCCESS: Unit test file appears to be valid JSONL format.

Attempting Hugging Face Hub login...


SUCCESS: Hugging Face login successful!

Checking for existence of Fine-tuned Adapters: /content/drive/MyDrive/266_fp_ph_z/llama3_8b_qa_finetuned_adapters_standard_hf
SUCCESS: Fine-tuned adapter files found.

Attempting to load Base Model: meta-llama/Meta-Llama-3-8B-Instruct...


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

SUCCESS: Base model and tokenizer loaded.

Checking for existence and attempting to load ChromaDB from: /content/drive/MyDrive/266_fp_ph_z/ph_z_rag/chroma_db
SUCCESS: ChromaDB found and loaded. Contains 508 documents.

--- All accessibility checks complete. ---


# Part 4: Execute RAG Inference in unit test questions, no passages with questions, generate answers.



*   Input: unit_test_passage_questions_clean.jsonl
*   Output: phase_z_gv_rag_generated_answers.jsonl



In [4]:
###Clan up RAM before the next run
import torch
import gc

print("Starting comprehensive GPU memory cleanup...")

# Step 1: Explicitly delete large objects and variables
try:
    del model
    del base_model
    del tokenizer
    # Delete any other large tensors or objects you may have
    # del your_large_tensor
except NameError:
    # This handles cases where the variables might not exist
    pass

# Step 2: Manually trigger the garbage collector
# This helps Python clean up objects that are no longer referenced
gc.collect()

# Step 3: Clear PyTorch's CUDA memory cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("GPU memory cleanup complete. Ready for the next run.")

Starting comprehensive GPU memory cleanup...
GPU memory cleanup complete. Ready for the next run.


In [5]:
##EXECUTE PHASE 2 = GENERATE RAG ANSWERS TO UNIT TEST QUESTIONS

import os
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from huggingface_hub import login
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from typing import Any, List
from google.colab import drive
from tqdm import tqdm

# --- Configuration ---
ROOT_DIR_ADAPTERS = "/content/drive/MyDrive/266_fp_ph_z"
FINE_TUNED_ADAPTERS_FOLDER_NAME = "llama3_8b_qa_finetuned_adapters_standard_hf"
FINE_TUNED_ADAPTERS_PATH = os.path.join(ROOT_DIR_ADAPTERS, FINE_TUNED_ADAPTERS_FOLDER_NAME)
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

CHROMA_DB_DIR = "/content/drive/MyDrive/266_fp_ph_z/ph_z_rag"
CHROMA_DB_PATH = os.path.join(CHROMA_DB_DIR, "chroma_db")

UNIT_TEST_FILE_NAME = "unit_test_passage_questions_clean_ph_z.jsonl"
UNIT_TEST_FILE_PATH = os.path.join(CHROMA_DB_DIR, UNIT_TEST_FILE_NAME)

GENERATED_ANSWERS_OUTPUT_FILE = os.path.join(CHROMA_DB_DIR, "phase_z_gv_rag_generated_answers.jsonl")


# --- Mount Google Drive (if not already mounted in this session) ---
print("Mounting Google Drive...")
if not os.path.exists('/content/drive/MyDrive'):
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully!")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        print("Please ensure you are running this in a Google Colab environment and authorize Drive access.")
        exit()
else:
    print("Google Drive already mounted.")

# --- Hugging Face Login (REQUIRED for Llama 3) ---
print("\nLogging into Hugging Face Hub...")
try:
    login()
    print("Hugging Face login successful!")
except Exception as e:
    print(f"Hugging Face login failed: {e}")
    print("Please ensure you have accepted the Llama 3 license and pasted a valid token.")
    exit()

# --- Load Base Model with 4-bit Quantization ---
print(f"\nLoading base model: {MODEL_NAME} with 4-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left" # For inference

print("Base model and tokenizer loaded.")

# --- Load Fine-tuned LoRA Adapters ---
print(f"Loading LoRA adapters from: {FINE_TUNED_ADAPTERS_PATH}...")
try:
    if not os.path.exists(os.path.join(FINE_TUNED_ADAPTERS_PATH, "adapter_config.json")) or \
       not os.path.exists(os.path.join(FINE_TUNED_ADAPTERS_PATH, "adapter_model.safetensors")):
        raise FileNotFoundError(f"Adapter files not found in {FINE_TUNED_ADAPTERS_PATH}")

    model = PeftModel.from_pretrained(base_model, FINE_TUNED_ADAPTERS_PATH)
    print("LoRA adapters loaded.")
    print("Merging LoRA adapters into base model for inference...")
    model = model.merge_and_unload()
    print("Adapters merged.")
except Exception as e:
    print(f"Error loading or merging LoRA adapters: {e}")
    print("Please ensure the fine-tuned adapters folder exists and contains 'adapter_config.json' and 'adapter_model.safetensors'.")
    exit()

model.eval()

# --- Initialize Embeddings and Vector Store for Retriever ---
print("\nInitializing embedding model and loading vector store for retriever...")
try:
    embedding_model_name = "BAAI/bge-small-en-v1.5"
    embeddings = SentenceTransformerEmbeddings(model_name=embedding_model_name)

    if not os.path.exists(CHROMA_DB_PATH) or not os.listdir(CHROMA_DB_PATH):
        raise FileNotFoundError(f"Chroma DB not found or is empty at {CHROMA_DB_PATH}. Please run Phase 1 first.")

    vectorstore = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    print("Retriever initialized.")
except Exception as e:
    print(f"Error initializing retriever or loading vector store: {e}")
    print("Please ensure the Chroma DB was successfully created/persisted in Phase 1.")
    exit()

# --- Define RAG Prompt Template ---
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based ONLY on the provided context. If the answer is not in the context, state that you don't know.\n\nContext: {context}"),
    ("user", "{input}")
])

# --- UPDATED: Standalone LLM Generation Function with warning removed ---
def generate_llm_response_from_messages(messages: Any) -> str:
    """
    Generates a response from the Llama 3 model given a list of chat messages.
    Handles the case where the messages are wrapped in an unexpected tuple.
    """
    if isinstance(messages, tuple) and len(messages) == 2 and messages[0] == 'messages':
        messages_list = messages[1]
    else:
        messages_list = messages

    formatted_messages = []
    for msg in messages_list:
        if hasattr(msg, 'type') and hasattr(msg, 'content'):
            formatted_messages.append({"role": msg.type, "content": msg.content})
        elif isinstance(msg, dict) and "role" in msg and "content" in msg:
            formatted_messages.append(msg)
        else:
            formatted_messages.append({"role": "user", "content": str(msg)})

    input_ids = tokenizer.apply_chat_template(
        formatted_messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=150,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
        )

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
    assistant_start_tag = "<|start_header_id|>assistant<|end_header_id|>\n"
    start_index = decoded_output.find(assistant_start_tag)

    if start_index != -1:
        generated_answer = decoded_output[start_index + len(assistant_start_tag):].strip()
        generated_answer = generated_answer.replace("<|eot_id|>", "").strip()
    else:
        generated_answer = "Could not parse assistant's response."
    return generated_answer


# --- Construct the RAG Chain using create_stuff_documents_chain ---
Youtube_chain = create_stuff_documents_chain(
    llm=generate_llm_response_from_messages,
    prompt=prompt
)
rag_chain = create_retrieval_chain(retriever, Youtube_chain)


# --- Load Questions from File and Perform Inference Individually ---
print(f"\nLoading questions from: {UNIT_TEST_FILE_PATH}")
questions_data = []
if not os.path.exists(UNIT_TEST_FILE_PATH):
    print(f"Error: Unit test file not found at {UNIT_TEST_FILE_PATH}.")
    print("Please ensure 'unit_test_passage_questions_clean.jsonl' is in your RAG data directory.")
    exit()

try:
    with open(UNIT_TEST_FILE_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            questions_data.append(json.loads(line))
    print(f"Loaded {len(questions_data)} questions from '{UNIT_TEST_FILE_NAME}'.")
except Exception as e:
    print(f"Error loading questions from {UNIT_TEST_FILE_PATH}: {e}")
    exit()

print(f"\nStarting RAG inference for {len(questions_data)} questions, processing individually...")
generated_results = []

with open(GENERATED_ANSWERS_OUTPUT_FILE, 'w', encoding='utf-8') as f_out:
    # Process each question one by one
    for i, entry in enumerate(tqdm(questions_data, desc="RAG Inference")):
        question = entry.get('question', 'No question provided.')
        doc_id = i

        try:
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            # Invoke the RAG chain on a single question
            response = rag_chain.invoke({"input": question})

            generated_answer = response.get('answer', 'No answer generated.')
            retrieved_chunks = response.get('context', [])
            num_chunks = len(retrieved_chunks)

            # Print the output in real-time
            print(f"\n--- Question {i+1} ---")
            print(f"Question: {question}")
            print(f"Retrieved Chunks: {num_chunks}")
            print(f"Generated Answer: {generated_answer}")

            # Store results for output file
            generated_results.append({
                "id": doc_id,
                "question": question,
                "generated_answer": generated_answer,
                "retrieved_chunks": num_chunks
            })
            json.dump(generated_results[-1], f_out)
            f_out.write('\n')

        except Exception as e:
            print(f"\n--- Question {i+1} (Error) ---")
            print(f"Question: {question}")
            print(f"An error occurred during RAG inference: {e}")
            generated_results.append({
                "id": doc_id,
                "question": question,
                "generated_answer": f"ERROR: {e}",
                "retrieved_chunks": 0
            })
            json.dump(generated_results[-1], f_out)
            f_out.write('\n')

print(f"\nPhase Z: RAG inference complete. All generated answers saved to: {GENERATED_ANSWERS_OUTPUT_FILE}")

Mounting Google Drive...
Google Drive already mounted.

Logging into Hugging Face Hub...


Hugging Face login successful!

Loading base model: meta-llama/Meta-Llama-3-8B-Instruct with 4-bit quantization...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Base model and tokenizer loaded.
Loading LoRA adapters from: /content/drive/MyDrive/266_fp_ph_z/llama3_8b_qa_finetuned_adapters_standard_hf...
LoRA adapters loaded.
Merging LoRA adapters into base model for inference...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


...


--- Question 1083 ---
Question: How does the passage describe the nature of Paramatma in relation to Jivi?
Retrieved Chunks: 3
Generated Answer: According to the passage, Paramatma (the supreme Lord) is the same as the Para Prakruthi, which is the universal power of energy that sustains everything. The passage describes Paramatma as eternal, true, and unaffected by change. It is also described as the unchanging Basis or Aadhara for all characteristics and Gunas (attributes) in the world.

In relation to Jivi (the individual self or living being), Paramatma is described as the Antharyamin (God spirit in all creation) who has penetrated into everything and sustains everything. The passage suggests that the Jivi is a part of Paramatma, and that the Jivi's nature and characteristics are dependent on Paramatma.


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
RAG Inference: 100%|█████████▉| 1084/1085 [1:46:24<00:06,  6.38s/it]


--- Question 1084 ---
Question: What does the statement about "effulgence of Brahmam" imply about individual perception of divinity?
Retrieved Chunks: 3
Generated Answer: The statement about the "effulgence of Brahman" implies that individual perception of divinity is limited. The passage suggests that people are bound by their defective and deceptive senses, which lead them to perceive the world as many and separate from the divine. It is only through the higher vision that one can see the unity and realize the divinity in all things.

In this context, the statement implies that individual perception of divinity is often clouded by ignorance and multiplicity, and that it is necessary to transcend these limitations in order to experience the bliss of the divine. The passage also suggests that this higher vision is not something that can be obtained through external means, but rather through inner realization and the recognition of the unity of all existence.

Therefore


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
RAG Inference: 100%|██████████| 1085/1085 [1:46:32<00:00,  5.89s/it]


--- Question 1085 ---
Question: How does this passage reflect on the relationship between ignorance and enlightenment?
Retrieved Chunks: 3
Generated Answer: The passage reflects on the relationship between ignorance and enlightenment by highlighting the role of ignorance (Ajnana) as the source of all types of sorrow. Krishna teaches Arjuna that ignorance is the delusion that one is the body, and that this identification is the root cause of all grief and misery. The passage suggests that ignorance is a fundamental obstacle to enlightenment, and that it must be overcome through the acquisition of right knowledge.

The passage also highlights the interconnectedness of ignorance and enlightenment, suggesting that ignorance is not just a lack of knowledge, but a fundamental distortion of reality. Krishna teaches that the distinction between good and bad, pleasure and pain, is an illusion created by ignorance, and that true understanding requires the recognition of the oneness of all

Phas